In [13]:
import os
from dotenv import load_dotenv
import langchain
from langchain_groq import ChatGroq
load_dotenv()

grop_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="Llama3-8b-8192", api_key=grop_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x35151c070>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x35151ce80>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [14]:
os.environ["HF_TOKEN"]= os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [16]:
# 1. Load, chunk and index the contents of the blog to create a retriever.

import bs4

loader = WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
                       bs_kwargs=dict(parse_only= bs4.SoupStrainer(class_=("post-content", "post-title", "post-header")
                           
                       )))
docs = loader.load()

In [17]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [27]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x3010f9660>)

In [32]:
retriever.invoke("what is reflection")

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Memory stream: is a long-term memory module (external database) that records a compreh

In [28]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [29]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [30]:
response=rag_chain.invoke({"input":"What is Self-Reflection"})
response

{'input': 'What is Self-Reflection',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Another quite distinct approach, LLM

In [31]:
response["answer"]

'Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

### Adding Chat History

In [33]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [34]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [35]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x3010f9660>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if 

In [48]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

chat_history.extend(
    [
        HumanMessage(content=question2),
        AIMessage(content=response2["answer"])
    ]
)

Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It helps agents to learn from their experiences and adjust their behavior accordingly.


In [49]:
chat_history

[HumanMessage(content='What is Self-Reflection'),
 AIMessage(content='Self-reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'),
 HumanMessage(content='Tell me more about it?'),
 AIMessage(content='Self-Reflection is a mechanism that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes. It helps agents to learn from their experiences and adjust their behavior accordingly.')]

In [50]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [51]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [53]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task Decomposition is a process where a complex task is broken down into smaller, more manageable steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts, which involve generating multiple thoughts or steps to solve a problem.'

In [54]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"I don't know. The provided context does not mention a specific task or problem that needs to be solved, so I'm unable to determine common ways of doing it."

In [66]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task decomposition is a technique used in planning and decision-making, where a complex task is broken down into smaller, more manageable subtasks or steps. This allows an agent, such as a machine learning model, to plan and solve the task more effectively by tackling one step at a time.'), HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task Decomposition is a process where a complex task is broken down into smaller, more manageable steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts, which involve generating multiple thoughts or steps to solve a problem.'), HumanMessage(content='What are common ways of doing it?'), AIMessage(content="I don't know. The provided context does not mention a specific task or problem that needs to be solved, so I'm unable to determine common ways of doing it.")])}

In [73]:
import pandas as pd

In [75]:
store["abc123"].messages

[HumanMessage(content='What is Task Decomposition?'),
 AIMessage(content='Task decomposition is a technique used in planning and decision-making, where a complex task is broken down into smaller, more manageable subtasks or steps. This allows an agent, such as a machine learning model, to plan and solve the task more effectively by tackling one step at a time.'),
 HumanMessage(content='What is Task Decomposition?'),
 AIMessage(content='Task Decomposition is a process where a complex task is broken down into smaller, more manageable steps. This is achieved by using techniques such as Chain of Thought (CoT) or Tree of Thoughts, which involve generating multiple thoughts or steps to solve a problem.'),
 HumanMessage(content='What are common ways of doing it?'),
 AIMessage(content="I don't know. The provided context does not mention a specific task or problem that needs to be solved, so I'm unable to determine common ways of doing it.")]

In [78]:
messages = store["abc123"].messages

In [81]:
human_messages = [message.content for message in messages if isinstance(message, HumanMessage)]

print(human_messages)

['What is Task Decomposition?', 'What is Task Decomposition?', 'What are common ways of doing it?']


In [83]:
ai_messages = [msg.content for msg in messages if isinstance(msg, AIMessage)]

# Ensure both lists are of the same length
min_length = min(len(human_messages), len(ai_messages))

# Create DataFrame
df = pd.DataFrame({
    'HumanMessage': human_messages[:min_length],
    'AIMessage': ai_messages[:min_length]
})

df

,HumanMessage,AIMessage
0,What is Task Decomposition?,Task decomposition is a technique used in plan...
1,What is Task Decomposition?,Task Decomposition is a process where a comple...
2,What are common ways of doing it?,I don't know. The provided context does not me...


In [84]:
min_length

3

In [108]:
import bs4

loader = WebBaseLoader(web_path=("https://hianime.to/home"),
                       bs_kwargs=dict(parse_only= bs4.SoupStrainer(class_=("film-name"), href = ("title")
                           
                       ))
                      
                           
                       )
docs = loader.load()

In [109]:
docs

[Document(metadata={'source': 'https://hianime.to/home'}, page_content='')]

In [110]:
import requests
from bs4 import BeautifulSoup

html_doc = requests.get("https://hianime.to/home")
soup = BeautifulSoup(html_doc.text, 'html.parser')

In [119]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>HiAnime Free Anime Streaming Homepage</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="index,follow" name="robots"/>
<meta content="en" http-equiv="content-language"/>
<meta content="Watch your favorite anime online in Dub or Sub format without registration on HiAnime.to fastest Streaming server NOW." name="description"/>
<meta content="watch anime online, anime site, free anime, anime to watch, online anime, anime streaming, stream anime online, english anime, english dubbed anime" name="keywords"/>
<meta content="website" property="og:type"/>
<meta content="https://hianime.to/home" property="og:url"/>
<meta content="HiAnime Free Anime Streaming Homepage" property="og:title"/>
<meta content="https://hianime.to/images/capture.png" property="og:image"/>
<meta content="650" property="og:image:width"/>
<meta content="350" property="og:image:height"/>
<meta content="Watch your favorite anime online i